In [10]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from scipy.stats import zscore
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel

In [11]:
def load_physiological_data(file_path):
    columns = ["time", "ECG", "BVP", "GSR", "Resp", "Skin_Temp", "EMG_z", "EMG_c", "EMG_t"]
    df = pd.read_csv(file_path, sep="\t", names=columns)
    return df

# Load valence-arousal data
def load_valence_arousal_data(file_path):
    df = pd.read_csv(file_path, sep="\t", names=["jsttime", "valence", "arousal"])
    return df

In [12]:
# def preprocess_data(physio_df, va_df, window_size=5):
#     # Downsample physiological data from 1000Hz to 1Hz
#     physio_downsampled = physio_df.iloc[::1000].copy()  # Simple decimation
    
#     # Downsample valence-arousal data from 20Hz to 1Hz
#     va_downsampled = va_df.iloc[::20].copy()
    
#     # Create windows for both datasets
#     start_time = min(physio_downsampled["time"].min(), va_downsampled["jsttime"].min())
    
#     physio_downsampled["window"] = ((physio_downsampled["time"] - start_time) // window_size).astype(int)
#     va_downsampled["window"] = ((va_downsampled["jsttime"] - start_time) // window_size).astype(int)
    
#     # Aggregate by window
#     physio_segmented = physio_downsampled.groupby("window").mean().reset_index()
#     va_segmented = va_downsampled.groupby("window").mean().reset_index()
    
#     return physio_segmented, va_segmented

def preprocess_data(physio_df, va_df, window_size=5):
    # Downsample
    physio_downsampled = physio_df.iloc[::1000].copy()
    va_downsampled = va_df.iloc[::20].copy()

    # Create window ID
    start_time = min(physio_downsampled["time"].min(), va_downsampled["jsttime"].min())
    physio_downsampled["window"] = ((physio_downsampled["time"] - start_time) // window_size).astype(int)
    va_downsampled["window"] = ((va_downsampled["jsttime"] - start_time) // window_size).astype(int)

    # Features to aggregate
    agg_dict = {
        "ECG": "mean",
        "BVP": ["mean", "std"],
        "GSR": ["mean", "std"],
        "Resp": "mean",
        "Skin_Temp": ["mean", "std"],
        "EMG_c": "mean",
        "EMG_t": "mean"
    }

    # Group by window
    physio_segmented = physio_downsampled.groupby("window").agg(agg_dict).reset_index()

    # Flatten column MultiIndex (e.g., ('BVP', 'mean') → 'BVP_mean')
    physio_segmented.columns = ["window"] + [f"{col}_{stat}" if stat else col for col, stat in physio_segmented.columns[1:]]

    # Rename columns to match model expectations
    physio_segmented = physio_segmented.rename(columns={
        "ECG_mean": "ECG",
        "BVP_mean": "BVP",
        "GSR_mean": "GSR",
        "Resp_mean": "Resp",
        "Skin_Temp_mean": "Skin_temp",
        "EMG_c_mean": "EMG_c",
        "EMG_t_mean": "EMG_t",
        "BVP_std": "BVP_std",
        "GSR_std": "GSR_std",
        "Skin_Temp_std": "Skin_temp_std"
    })

    # Compute slope for GSR (difference between successive means)
    physio_segmented["GSR_slope"] = physio_segmented["GSR"].diff().fillna(0)

    # Same for valence-arousal
    va_segmented = va_downsampled.groupby("window").mean().reset_index()

    return physio_segmented, va_segmented



In [13]:
def compute_rulsif_change_scores(X, alpha=0.1, sigma=0.1, lambda_param=1e-3):
    n = len(X) - 1
    change_scores = np.zeros(n)
    
    for i in range(n):
        X_t, X_t_next = X[i], X[i + 1]
        
        # Compute Gaussian Kernel Matrix
        K_t = rbf_kernel(X_t.reshape(-1, 1), X_t.reshape(-1, 1), gamma=1/(2*sigma**2))
        K_t_next = rbf_kernel(X_t_next.reshape(-1, 1), X_t_next.reshape(-1, 1), gamma=1/(2*sigma**2))
        
        # Compute Weights using Least Squares Importance Fitting (LSIF)
        H = alpha * K_t + (1 - alpha) * K_t_next + lambda_param * np.eye(K_t.shape[0])
        h = np.mean(K_t, axis=1)
        
        theta = np.linalg.solve(H, h)
        
        # Compute Change Score
        change_scores[i] = np.mean(np.square(K_t_next.dot(theta) - 1))
    
    return change_scores

# Identify significant changes and label opportune moments
def label_opportune_moments(change_scores):
    mean, std = np.mean(change_scores), np.std(change_scores)
    threshold = mean + 3 * std
    outliers = change_scores > threshold
    
    # Clustering the remaining scores
    valid_indices = np.where(~outliers)[0]  # Indices of non-outliers
    valid_scores = change_scores[valid_indices]
    
    if len(valid_scores) > 1:  # Ensure there are enough points for clustering
        kmeans = KMeans(n_clusters=2, random_state=42).fit(valid_scores.reshape(-1, 1))
        centroids = kmeans.cluster_centers_.flatten()
        high_cluster = np.argmax(centroids)
        high_values = (kmeans.labels_ == high_cluster) & (valid_scores > centroids[high_cluster])
        
        # Map high_values back to the original indices
        high_values_original = np.zeros_like(change_scores, dtype=bool)
        high_values_original[valid_indices] = high_values
    else:
        # If there are not enough valid scores, treat all as non-opportune
        high_values_original = np.zeros_like(change_scores, dtype=bool)
    
    # Mark opportune moments
    opportune_moments = np.where(outliers | high_values_original)[0]
    return opportune_moments

In [14]:
# Align valence-arousal data with physiological data windows
def align_data(physiological_data, valence_arousal_data, window_size=5):
    # Convert jsttime to window index
    start_time = physiological_data["time"].min()
    valence_arousal_data["window"] = ((valence_arousal_data["jsttime"] - start_time) // window_size).astype(int)
    
    # Merge data on window index
    merged_data = pd.merge(physiological_data, valence_arousal_data, on="window", how="inner")
    return merged_data

def scale_valence_arousal(df):
    df = df.copy()
    for dim in ["valence", "arousal"]:
        max_abs = np.max(np.abs(df[dim]))
        df[dim] = df[dim] / max_abs if max_abs != 0 else 0.0
    return df

# Map valence-arousal to emotion classes
def map_to_emotion_classes(valence, arousal):
    # Define emotion classes based on valence and arousal
    if valence >= 0 and arousal >= 0:
        return "Happy"
    elif valence >= 0 and arousal < 0:
        return "Relaxed"
    elif valence < 0 and arousal >= 0:
        return "Stressed"
    else:
        return "Sad"

In [15]:
def train_and_evaluate_classifier(X_train, X_test, y_train, y_test):
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = model.predict(X_test)
    
    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    report = print(classification_report(y_test, y_pred, zero_division=0))

    print("Classification Report:")
    print(report)
    
    return accuracy, f1

In [16]:
from tensorflow.keras.layers import Input, LSTM, Dense, Activation, Dropout, Bidirectional, Multiply, Flatten, Lambda,RepeatVector, Permute
from tensorflow.keras import backend as K 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

def elliott(x, p):
    return (p * x) / (1 + K.abs(x))

# Derivative of PEF
def elliott_derivative(x, p):
    return p / ((K.abs(x) + 1) ** 2)

# Swish Activation Function
def swish(x):
    return x * K.sigmoid(x)

# Custom PEF Activation Layer
class PEFLayer(Activation):
    def __init__(self, activation, **kwargs):
        super(PEFLayer, self).__init__(activation, **kwargs)
        self.p = K.variable(1.0)  # Initialize parameter p

    def call(self, inputs):
        return elliott(inputs, self.p)

In [17]:
from tensorflow.keras.models import load_model
import tensorflow as tf

def load_student_model(user_id):
    model_path = f"saved_models/user_User{user_id}/student_model"
    return tf.keras.models.load_model(model_path, custom_objects={'PEFLayer': PEFLayer, 'elliott': elliott, 'swish': swish})

In [ ]:
if __name__ == "__main__":
    results = []

    opportune_features = ['BVP', 'BVP_std',
            'GSR', 'GSR_std', 'GSR_slope',
            'Skin_temp', 'Skin_temp_std']
    model_features = ["ECG", "EMG_c", "EMG_t", "Resp", "Skin_temp"]
    student_features = ['BVP', 'BVP_std',
            'GSR', 'GSR_std', 'GSR_slope',
            'Skin_temp', 'Skin_temp_std']

    threshold = 0.5 

    for user_id in range(1, 31):
        try:
            physio_data = load_physiological_data(
                f"case_dataset-master/case_dataset-master/data/raw/physiological/sub{user_id}_DAQ.txt")
            va_data = load_valence_arousal_data(
                f"case_dataset-master/case_dataset-master/data/raw/annotations/sub{user_id}_joystick.txt")

            physio_segmented, va_segmented = preprocess_data(physio_data, va_data)
            va_segmented = scale_valence_arousal(va_segmented)
            merged_data = pd.merge(physio_segmented, va_segmented, on="window", how="inner")

            merged_data["emotion"] = merged_data.apply(
                lambda row: map_to_emotion_classes(row["valence"], row["arousal"]), axis=1
            )

            # Baseline model
            X_model = merged_data[model_features]
            y = merged_data["emotion"]
            X_train, X_test, y_train, y_test = train_test_split(X_model, y, test_size=0.2, random_state=42)

            accuracy_all, f1_all = train_and_evaluate_classifier(X_train, X_test, y_train, y_test)

            # RuLSIF-based change point detection
            X_opportune = merged_data[opportune_features].apply(zscore).values
            change_scores = compute_rulsif_change_scores(X_opportune)
            opportune_moments = label_opportune_moments(change_scores)

            if len(opportune_moments) < 5:
                print(f"Too few opportune moments for User {user_id}. Skipping...")
                continue

            opportune_data = merged_data[merged_data["window"].isin(opportune_moments)]
            inopportune_data = merged_data[~merged_data["window"].isin(opportune_moments)]

            # Split opportune_data (60% for training detection, 40% held for test)
            opportune_train_data, opportune_test_data = train_test_split(
                opportune_data, test_size=0.2, random_state=42)

            detection_input_data = pd.concat([opportune_train_data, inopportune_data])
            student_signal_data = detection_input_data[student_features]

            # Compute change scores
            student_change_scores = compute_rulsif_change_scores(student_signal_data.values)
            student_change_scores = np.append(student_change_scores, 0)  # match length

            # Combine raw + change (shape: [num_samples, features])
            student_input_array = np.stack([
                student_signal_data["BVP"].values,
                student_signal_data["BVP_std"].values,
                student_signal_data["GSR"].values,
                student_signal_data["GSR_std"].values,
                student_signal_data["GSR_slope"].values,
                student_signal_data["Skin_temp"].values,
                student_signal_data["Skin_temp_std"].values,
                student_change_scores
            ], axis=1)

            # Expand to 3D shape (samples, timesteps=1, features=8)
            student_input_3D = np.expand_dims(student_input_array, axis=1)

            # Predict with student model

            student_model = load_student_model(user_id)
            _, _, pred_probs = student_model.predict(student_input_3D, verbose=0)
            predicted_opportune_mask = (pred_probs.flatten() > 0.5)
            detected_opportune_data = detection_input_data[predicted_opportune_mask]

            if len(detected_opportune_data) < 5:
                print(f"Too few predicted opportune moments for User {user_id}. Skipping...")
                continue

            # Emotion classification: Train on detected opportune, test on held-out RuLSIF-opportune
            X_train_emotion = detected_opportune_data[model_features]
            y_train_emotion = detected_opportune_data["emotion"]
            X_test_emotion = opportune_test_data[model_features]
            y_test_emotion = opportune_test_data["emotion"]

            accuracy_opportune, f1_opportune = train_and_evaluate_classifier(
                X_train_emotion, X_test_emotion, y_train_emotion, y_test_emotion
            )

            results.append({
                "User ID": user_id,
                "Accuracy (All)": accuracy_all,
                "F1 Score (All)": f1_all,
                "Accuracy (Opportune)": accuracy_opportune,
                "F1 Score (Opportune)": f1_opportune,
                "Opportune Moments Count": len(opportune_moments)
            })

            print(f"Successfully processed User {user_id}")
            print("\n\n\n")

        except FileNotFoundError:
            print(f"Data files for User {user_id} not found. Skipping...")
            continue
        except Exception as e:
            print(f"Unexpected error processing User {user_id}: {str(e)}. Skipping...")
            continue

    results_df = pd.DataFrame(results)
    results_df.to_csv("emotion_classification_results_robust.csv", index=False)
    print("\nFinal Results:")
    print(results_df)


              precision    recall  f1-score   support

       Happy       0.84      0.97      0.90        73
     Relaxed       0.67      0.29      0.40        14
         Sad       0.00      0.00      0.00         1
    Stressed       0.88      0.64      0.74        11

    accuracy                           0.83        99
   macro avg       0.59      0.47      0.51        99
weighted avg       0.81      0.83      0.80        99

Classification Report:
None


c:\Users\Harshit Kumar\.conda\envs\tf-gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


              precision    recall  f1-score   support

       Happy       0.77      0.94      0.85        18
     Relaxed       0.00      0.00      0.00         2
    Stressed       0.88      0.70      0.78        10

    accuracy                           0.80        30
   macro avg       0.55      0.55      0.54        30
weighted avg       0.76      0.80      0.77        30

Classification Report:
None
Successfully processed User 1




              precision    recall  f1-score   support

       Happy       0.90      0.93      0.92        58
     Relaxed       0.82      0.82      0.82        22
         Sad       0.00      0.00      0.00         1
    Stressed       0.88      0.83      0.86        18

    accuracy                           0.88        99
   macro avg       0.65      0.65      0.65        99
weighted avg       0.87      0.88      0.87        99

Classification Report:
None


c:\Users\Harshit Kumar\.conda\envs\tf-gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


              precision    recall  f1-score   support

       Happy       0.85      1.00      0.92        17
     Relaxed       1.00      0.50      0.67         2
         Sad       1.00      1.00      1.00         1
    Stressed       1.00      0.85      0.92        13

    accuracy                           0.91        33
   macro avg       0.96      0.84      0.88        33
weighted avg       0.92      0.91      0.91        33

Classification Report:
None
Successfully processed User 2




              precision    recall  f1-score   support

       Happy       0.79      0.89      0.84        63
     Relaxed       0.76      0.57      0.65        23
         Sad       0.00      0.00      0.00         1
    Stressed       0.80      0.67      0.73        12

    accuracy                           0.78        99
   macro avg       0.59      0.53      0.55        99
weighted avg       0.78      0.78      0.77        99

Classification Report:
None


...

              precision    recall  f1-score   support

       Happy       0.92      1.00      0.96        23
     Relaxed       0.00      0.00      0.00         2
         Sad       0.67      0.67      0.67         3

    accuracy                           0.89        28
   macro avg       0.53      0.56      0.54        28
weighted avg       0.83      0.89      0.86        28

Classification Report:
None
Successfully processed User 28




              precision    recall  f1-score   support

       Happy       0.89      0.99      0.94        85
     Relaxed       1.00      0.33      0.50         9
    Stressed       0.50      0.20      0.29         5

    accuracy                           0.89        99
   macro avg       0.80      0.51      0.57        99
weighted avg       0.88      0.89      0.87        99

Classification Report:
None


c:\Users\Harshit Kumar\.conda\envs\tf-gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


              precision    recall  f1-score   support

       Happy       0.84      0.93      0.88        28
     Relaxed       1.00      1.00      1.00         1
         Sad       0.00      0.00      0.00         1
    Stressed       0.33      0.20      0.25         5

    accuracy                           0.80        35
   macro avg       0.54      0.53      0.53        35
weighted avg       0.75      0.80      0.77        35

Classification Report:
None
Successfully processed User 29




              precision    recall  f1-score   support

       Happy       0.82      0.99      0.90        67
     Relaxed       0.90      0.64      0.75        14
         Sad       0.00      0.00      0.00         1
    Stressed       0.88      0.41      0.56        17

    accuracy                           0.83        99
   macro avg       0.65      0.51      0.55        99
weighted avg       0.84      0.83      0.81        99

Classification Report:
None


c:\Users\Harshit Kumar\.conda\envs\tf-gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


              precision    recall  f1-score   support

       Happy       0.71      0.75      0.73        16
     Relaxed       1.00      0.14      0.25         7
    Stressed       0.36      0.67      0.47         6

    accuracy                           0.59        29
   macro avg       0.69      0.52      0.48        29
weighted avg       0.71      0.59      0.56        29

Classification Report:
None
Successfully processed User 30





Final Results:
    User ID  Accuracy (All)  F1 Score (All)  Accuracy (Opportune)  \
0         1        0.828283        0.801140              0.800000   
1         2        0.878788        0.873872              0.909091   
2         4        0.797980        0.759483              0.866667   
3         5        0.737374        0.713089              0.774194   
4         8        0.919192        0.916678              0.903226   
5        10        0.787879        0.772916              0.814815   
6        13        0.797980        0.774772              